<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Debugging" data-toc-modified-id="Debugging-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Debugging</a></span></li><li><span><a href="#Entry-price" data-toc-modified-id="Entry-price-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Entry price</a></span></li><li><span><a href="#Position-value" data-toc-modified-id="Position-value-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Position value</a></span></li><li><span><a href="#Margin-available" data-toc-modified-id="Margin-available-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Margin available</a></span></li></ul></div>

# Debugging


    %debug func(*args)           # line magic
    ipdb> s                      # step into func
    ipdb> b line_num             # set breakpoint
    ipdb> c                      # continue until breakpoint
    ipdb> b                      # list all breakpoint
    ipdb> clear breakpoint_num   # remove breakpoint_num
    ipdb> c                      # continue untill function finishes
    ipdb> u line_num             # run until line_num
    ipdb> q                      # quit debugger
    
<u>**To debug on a specific date**</u>:
 
```
class DummyStrategy(Strategy):

    def init(self):
        pass
        
    def next(self):
        if self.data.index[-1] == pd.Timestamp('2019-10-18'):
            breakpoint()
```

In [1]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, SignalStrategy, TrailingStrategy
from backtesting.test import SMA

import pandas_datareader.data as dtr
import requests_cache
import datetime
expire_after = datetime.timedelta(days = 7)
session = requests_cache.CachedSession(cache_name='cache', backend='sqlite',
                                       expire_after=expire_after)

tickers = ["LNTA.ME", "MTSS.ME","GAZP.ME","CHMF.ME","SBER.ME"]

df = pd.DataFrame()
for ticker in tickers:
    try:
        df_ = dtr.DataReader(ticker, data_source="yahoo" , session=session, retry_count=1)
        df_.insert(0, "ticker", ticker)
    except:
        print(f"ticker '{ticker}' failed")
        pass
    df = df.append(df_)

GAZP = df[df.ticker == "GAZP.ME"]
GAZP.head()

/home/sergey/anaconda3/lib/python3.7/site-packages/backtesting/_plotting.py:45: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

,ticker,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,,
2015-12-21,GAZP.ME,133.940002,131.000000,132.000000,131.500000,28393100.0,131.500000
2015-12-22,GAZP.ME,133.539993,131.809998,132.000000,133.199997,20022420.0,133.199997
2015-12-23,GAZP.ME,135.300003,133.350006,133.550003,135.300003,26760910.0,135.300003
2015-12-24,GAZP.ME,138.080002,134.449997,135.850006,136.000000,36828650.0,136.000000
2015-12-25,GAZP.ME,136.490005,134.029999,136.490005,134.479996,14866310.0,134.479996


In [2]:
class SmaSignalTrailing(SignalStrategy, TrailingStrategy):
    
    n_fast = 24
    n_slow = 55
    trailing_sl = 3
    sz = .95
    

    def init(self):
        super().init()
        nfast = self.I(SMA, self.data.Close, self.n_fast)
        nslow = self.I(SMA, self.data.Close, self.n_slow)
        signal = (pd.Series(nfast) > nslow).astype(int).diff().fillna(0)
        signal = signal.replace(-1,0)
        self.set_signal(entry_size = self.sz*signal)
        self.set_trailing_sl(self.trailing_sl)
        
bt = Backtest(GAZP, SmaSignalTrailing, commission=.002,margin=.5)

# Entry price  

>    - `self._data.Open[-1]*1.002` or,   
>    - if trade on close `self.last_price`, which is `self._data.Close[-2]`,   
>    - multiplied by `1 + commisions` for long and `1 - commisions` for short
>
# Position value  

>    - `self.trades[0].size * self.last_price`
>
# Margin available  

>    - available cash is stored in `self.margin_available`
>    - calsulated as:
>        - `self.equity - (self.trades[0].value / self._leverage)`
>        - equity is: `self._cash + self.trades[0].pl` 

In [17]:
%%debug
bt.run()

NOTE: Enter 'c' at the ipdb>  prompt to continue execution.
> <string>(2)<module>()

ipdb> s
--Call--
> /home/sergey/anaconda3/lib/python3.7/site-packages/backtesting/backtesting.py(1088)run()
   1086         self._results = None
   1087 
-> 1088     def run(self, **kwargs) -> pd.Series:
   1089         """
   1090         Run the backtest. Returns `pd.Series` with results and statistics.

ipdb> b 853
Breakpoint 1 at /home/sergey/anaconda3/lib/python3.7/site-packages/backtesting/backtesting.py:853
ipdb> c
> /home/sergey/anaconda3/lib/python3.7/site-packages/backtesting/backtesting.py(853)_process_orders()
    851             # precompute true size in units, accounting for margin and spread/commissions
    852             size = order.size
1-> 853             if -1 < size < 1:
    854                 #breakpoint()
    855                 size = copysign(int((self.margin_available * self._leverage * abs(size))

ipdb> self._data.Open[-1]*1.002
136.4423381652832
ipdb> self.equity
10000
ipd

ipdb> self.trades[0].pl
1275.694316314699
ipdb> self.trades[0].value - self.trades[0].entry_price*self.trades[0].size
1275.6943163147007
ipdb> self.trades[0].size*self.last_price - self.trades[0].entry_price*self.trades[0].size
1275.6943163147007
ipdb> self.trades[0].size*(self.last_price - self.trades[0].entry_price)
1275.694316314699
ipdb> self.trades[0].pl
1275.694316314699
ipdb> self.equity
11275.694316314699
ipdb> self.cash
*** AttributeError: '_Broker' object has no attribute 'cash'
ipdb> self._cash
10000
ipdb> self._cash + self.trades[0].pl
11275.694316314699
ipdb> self.equity
11275.694316314699
ipdb> dir(self)
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_adjusted_price', '_cash', '

In [4]:
stats = bt.run()
stats._trades.sort_values("EntryTime")

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
1,139,200,285,136.442338,146.646100,1418.322896,0.074784,2016-10-10,2017-02-08,121 days
0,15,221,285,138.276000,146.646100,125.551500,0.060532,2016-11-09,2017-02-08,91 days
4,178,354,535,122.845198,138.046109,2705.762103,0.123740,2017-05-23,2018-02-08,261 days
3,2,403,535,117.053640,138.046109,41.984939,0.179341,2017-08-01,2018-02-08,191 days
2,7,443,535,123.496500,138.046109,101.847264,0.117814,2017-09-26,2018-02-08,135 days
5,187,583,621,145.490397,138.676099,-1274.273755,-0.046837,2018-04-19,2018-06-15,57 days
7,175,658,710,141.632706,159.806104,3180.344572,0.128314,2018-08-07,2018-10-18,72 days
6,12,671,710,141.783000,159.806104,216.277244,0.127118,2018-08-24,2018-10-18,55 days
9,207,759,795,151.302000,153.096097,371.378070,0.011858,2018-12-27,2019-02-21,56 days
8,24,778,795,160.420206,153.096097,-175.778620,-0.045656,2019-01-29,2019-02-21,23 days


In [14]:
GAZP["2016-11"]

,ticker,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,,
2016-11-01,GAZP.ME,142.440002,139.009995,139.119995,141.979996,49424750.0,141.979996
2016-11-02,GAZP.ME,143.639999,140.029999,141.490005,140.809998,43150830.0,140.809998
2016-11-03,GAZP.ME,141.300003,139.300003,140.500000,139.899994,24879410.0,139.899994
2016-11-07,GAZP.ME,140.960007,138.500000,140.500000,138.500000,23567564.0,138.500000
2016-11-08,GAZP.ME,140.979996,138.809998,138.860001,139.320007,20844700.0,139.320007
2016-11-09,GAZP.ME,145.619995,137.550003,138.000000,145.619995,79385300.0,145.619995
2016-11-10,GAZP.ME,151.300003,145.750000,146.449997,147.399994,103218470.0,147.399994
2016-11-11,GAZP.ME,149.500000,145.789993,147.399994,147.880005,45422659.0,147.880005
2016-11-14,GAZP.ME,149.669998,145.960007,148.600006,146.259995,31655630.0,146.259995
